In [1]:
from pathlib import Path
import numpy as np
from scipy.optimize import curve_fit

from astropy.table import Table, vstack
from astropy.coordinates import SkyCoord

%matplotlib inline
import matplotlib.pyplot as plt

from stellarphot.utils.magnitude_transforms import get_cat, f, opts_to_str, calc_residual, transform_to_catalog

# Parameters

The magnitudes in each image are fit using this model:

$$
r_{p, c} = a r_{p, inst} + b r_{p, inst}^2 + c (B_c - V_c) + d (B_c - V_c)^2 + z
$$

The parameters in the cell below set the range of values the fit is constrained to. The way to fix a parameter is to give it a very, very small range for the constraint.

More specifically, each of the fit values is subject to these constraints:

+ $1 - a_{delta} < a < 1 + a_{delta}$
+ $b_{min} < b < -b_{min}$
+ $c_{min} < c < -c_{min}$
+ $d_{min} < d < -d_{min}$
+ The range for the zero point is $18 < z < 22$.


`output_dir` is where the PNG and FITS files generated by this notebook are stored. `run_name` is a descriptive name for the settings you have chosen that gets included in the output file names.

### *Recommendation:*

+ Keep $b$ essentially fixed. 
+ Fixing $d$ is ok for now too, I think.

In both cases, setting a min of `1e-6` or something should do the trick.


In [18]:
a_delta = 0.5
b_min = -0.1
c_min = -0.5
d_min = -1e-6

our_filters = ['SI']

aavso_band_names = dict(
    B='B', 
    V='V',
    gp='SG',
    rp='SR',
    ip='SI',
    SI='SI'
)

cat_color_colums = dict(
    B=('Bmag', 'Vmag'),
    V=('Bmag', 'Vmag'),
    gp=('g_mag', 'r_mag'),
    rp=('r_mag', 'i_mag'),
    ip=('r_mag', 'i_mag'),
    SI=('r_mag', 'i_mag'),
)

cat_filter = dict(
    B='Bmag', 
    V='Vmag',
    gp='g_mag',
    rp='r_mag',
    ip='i_mag',
    SI='i_mag',
)

input_photometry_file = 'TIC-402828941-2021-09-28.ecsv'
output_photometry_file = 'some_name.ecsv'

In [9]:
all_mags = Table.read(input_photometry_file)

# Ensure we have the right table ordering later
all_mags.sort(['passband', 'bjd'])

## Get ready for the transform

In [13]:
filter_groups = all_mags.group_by('passband')


# Check: do we have any unexpected filters?

assert all(k[0] in cat_filter.keys() for k in filter_groups.groups.keys)

## Do the transforms, one filter at a time

In [19]:
output_table = [] 

for k, group in zip(our_filters, filter_groups.groups):
    print(f'Transforming band {k}')
    by_bjd = group.group_by('bjd')
    
    transform_to_catalog(by_bjd, f'mag_inst', aavso_band_names[k], 
                     obs_error_column='mag_error', 
                     zero_point_range=[12, 25],
                     c_delta=0.5, # b_delta=0.1, 
                     cat_filter=cat_filter[k], cat_color=cat_color_colums[k],
                     in_place=True);
    output_table.append(by_bjd.copy())

output_table = vstack(output_table, join_type='outer')

Transforming band SI


KeyError: 'RA'

In [ ]:
plt.plot(by_bjd['mag_inst_cal'], by_bjd['mag_cat'], 'o')
plt.grid()

In [ ]:
plt.plot(by_bjd['mag_cat'] - by_bjd['mag_inst_cal'], 'o')
plt.grid()

In [ ]:
output_table.write(output_photometry_file)